# Es necesario informar el días desde que se van a recoger datos y el número de dias, así como el código del aeropueto
Buscar aquí: https://www.wunderground.com/history

In [1]:
#Parámetros

#aeropuerto = 'LIPC' #Cervia
#aeropuerto = 'LEMG' #Marbella
#aeropuerto = 'EDDF' #Frankfurt
#aeropuerto = 'LEPA' #Palma
#aeropuerto = 'GCRR' #Lanzaorte
#aeropuerto = 'LEVT' #Vitoria
#aeropuerto = 'LELC' #Murcia. San Javier
#aeropuerto = 'EDDS' #Nuremberg
aeropuerto = 'LEMD' #MadridBarajas


anio_ini = 2017
mes_ini = 9
dia_ini = 1
num_dias = 30

anios = 5

In [2]:
#Dependecias Externas

import sys
import requests
from bs4 import BeautifulSoup
import xlsxwriter
import pandas as pd
import time
import datetime


In [3]:
#Definición de Funciones

def lista_fecha(anio, mes, dia, dias):
    datelist = pd.date_range(datetime.datetime(anio, mes, dia), periods=dias).tolist()
    result = map(lambda x: x.to_pydatetime().date().strftime('%Y/%m/%d'), datelist)
    return list(result)

def leer_meteo(url):
    headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_5)",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "accept-charset": "ISO-8859-1,utf-8;q=0.7,*;q=0.3",
    "accept-encoding": "gzip,deflate,sdch",
    "accept-language": "en-US,en;q=0.8",
    }

    #Metemos las cookies para que el html que devuelva sea el que necesitamos
    cook = dict(Prefs='FAVS:1|WXSN:1|PWSOBS:1|WPHO:1|PHOT:1|RADC:0|RADALL:0|HIST0:NULL|GIFT:1|SHOWMETAR:1|')
    
    r = requests.get(url, headers=headers, cookies=cook)
    
    if r.status_code != 200:
        print("request denied")
        print(r.status_code)
        sigue = 0

    soup = BeautifulSoup(r.text, 'html.parser')
    lista_info = []
    try:
        table =  soup.select("#obsTable")[0]
        for tr in table.find_all('tr'):
            for tds in tr.find_all('td', attrs={"colspan":"12"}):
                lista_info.append(tds.text)
                #print(tds.text)
        return lista_info
    except:
        pass

def filtrar_metars (lista):
    result = []
    try:
        lista.sort()
    
        for linea in lista:
            if linea[:5] == 'METAR':
                result.append(linea)
        return result
    except:
        pass

def metar_parser(solo_metars):
    from metar import Metar
    metar = []
    try:
        for element in solo_metars:
            lluvia = 0
            dir_viento = 0
            fuerza_viento = 0
            temp = 0
            # Initialize a Metar object with the data from solo_metars
            try:
                obs = Metar.Metar(element)
            except:
                pass

            estacion = aeropuerto

            try:
                h = str(obs.time.hour)
                m = "00"+str(obs.time.minute)
                hora = dia+" "+h+":"+m[-2:]
            except:
                hora = ''

            try:
                temp = obs.temp.value('C')                  
            except:
                temp = ''

            try:
                dir_viento = obs.wind_dir.value()
            except:
                dir_viento = ''

            try:
                fuerza_viento = obs.wind_speed.value("KMH")
            except:
                fuerza_viento = ''

            try:
                lluvia = obs.precip_1hr.string("in")/0.061024
            except:
                lluvia = ''

            metar.append([estacion, hora, temp, fuerza_viento, dir_viento, lluvia])
        return metar
    except:
        pass


In [4]:
#Ejecucuón

lista_df =[]
for year in range(anios):
    for dia in lista_fecha(anio_ini-year,mes_ini,dia_ini,num_dias):
        print(dia)
        url = 'https://www.wunderground.com/history/airport/'+aeropuerto+'/'+dia+'/DailyHistory.html'
        print(url)
        meteo_list = leer_meteo(url)
        solo_metars = filtrar_metars(meteo_list)
        info = metar_parser(solo_metars)
        dfinfo = pd.DataFrame(info)
        try:
            dfinfo.columns = ['Estacion','Fecha_Hora','Temp','Fuerza_Viento','Direccion_Viento','Lluvia']
            dfinfo['Fecha_Hora'] = pd.to_datetime(dfinfo['Fecha_Hora'])
        except:
            pass
        
        lista_df.append(dfinfo)
print('fin bucle - guardando')
dfresultado_final = pd.concat(lista_df)
print('guardado - printing')
dfresultado_final.to_excel("metars"+aeropuerto+".xls")
print('fin')


2017/09/01
https://www.wunderground.com/history/airport/LEMD/2017/09/01/DailyHistory.html
2017/09/02
https://www.wunderground.com/history/airport/LEMD/2017/09/02/DailyHistory.html
2017/09/03
https://www.wunderground.com/history/airport/LEMD/2017/09/03/DailyHistory.html
2017/09/04
https://www.wunderground.com/history/airport/LEMD/2017/09/04/DailyHistory.html
2017/09/05
https://www.wunderground.com/history/airport/LEMD/2017/09/05/DailyHistory.html
2017/09/06
https://www.wunderground.com/history/airport/LEMD/2017/09/06/DailyHistory.html
2017/09/07
https://www.wunderground.com/history/airport/LEMD/2017/09/07/DailyHistory.html
2017/09/08
https://www.wunderground.com/history/airport/LEMD/2017/09/08/DailyHistory.html
2017/09/09
https://www.wunderground.com/history/airport/LEMD/2017/09/09/DailyHistory.html
2017/09/10
https://www.wunderground.com/history/airport/LEMD/2017/09/10/DailyHistory.html
2017/09/11
https://www.wunderground.com/history/airport/LEMD/2017/09/11/DailyHistory.html
2017/09/12

2014/09/03
https://www.wunderground.com/history/airport/LEMD/2014/09/03/DailyHistory.html
2014/09/04
https://www.wunderground.com/history/airport/LEMD/2014/09/04/DailyHistory.html
2014/09/05
https://www.wunderground.com/history/airport/LEMD/2014/09/05/DailyHistory.html
2014/09/06
https://www.wunderground.com/history/airport/LEMD/2014/09/06/DailyHistory.html
2014/09/07
https://www.wunderground.com/history/airport/LEMD/2014/09/07/DailyHistory.html
2014/09/08
https://www.wunderground.com/history/airport/LEMD/2014/09/08/DailyHistory.html
2014/09/09
https://www.wunderground.com/history/airport/LEMD/2014/09/09/DailyHistory.html
2014/09/10
https://www.wunderground.com/history/airport/LEMD/2014/09/10/DailyHistory.html
2014/09/11
https://www.wunderground.com/history/airport/LEMD/2014/09/11/DailyHistory.html
2014/09/12
https://www.wunderground.com/history/airport/LEMD/2014/09/12/DailyHistory.html
2014/09/13
https://www.wunderground.com/history/airport/LEMD/2014/09/13/DailyHistory.html
2014/09/14

### Pruebas

In [79]:
from metar import Metar

for element in solo_metars:
    lluvia = 0
    # Initialize a Metar object with the data from solo metars
    obs = Metar.Metar(element)

    # The 'station_id' attribute is a string.
    estacion = obs.station_id
    #print("station: %s" % obs.station_id)

    #if obs.type:
    #  print("type: %s" % obs.report_type())

    # The 'time' attribute is a datetime object
    if obs.time:
        hora = obs.time.ctime()       
        #print("time: %s" % obs.time.ctime())

    # The 'temp' and 'dewpt' attributes are temperature objects
    if obs.temp:
        temp = obs.temp.value('C')                  
        #print("temperature: %s" % obs.temp.string("C"))

    #if obs.dewpt:
    #  print("dew point: %s" % obs.dewpt.string("C"))

    # The wind() method returns a string describing wind observations
    # which may include speed, direction, variability and gusts.
    if obs.wind_dir:
        dir_viento = obs.wind_dir.value()
    
    if obs.wind_speed:
        fuerza_viento = obs.wind_speed.value("KMH")
    #print("wind: %s" % obs.wind())

    # The peak_wind() method returns a string describing the peak wind 
    # speed and direction.
    #if obs.wind_speed_peak:
    #    fuerza_viento_pico = obs.peak_wind()
    #print("wind: %s" % obs.peak_wind())

    # The visibility() method summarizes the visibility observation.
    #if obs.vis:
    #  print("visibility: %s" % obs.visibility())

    # The runway_visual_range() method summarizes the runway visibility
    # observations.
    #if obs.runway:
    #  print("visual range: %s" % obs.runway_visual_range())

    # The 'press' attribute is a pressure object.
    #if obs.press:
    #  print("pressure: %s" % obs.press.string("mb"))

    # The 'precip_1hr' attribute is a precipitation object.
    if obs.precip_1hr:
        lluvia = obs.precip_1hr.string("in")
    #print("precipitation: %s" % obs.precip_1hr.string("in"))

    # The present_weather() method summarizes the weather description (rain, etc.)
    #print("weather: %s" % obs.present_weather())

    # The sky_conditions() method summarizes the cloud-cover observations.
    #print("sky: %s" % obs.sky_conditions("\n     "))

    # The remarks() method describes the remark groups that were parsed, but 
    # are not available directly as Metar attributes.  The precipitation, 
    # min/max temperature and peak wind remarks, for instance, are stored as
    # attributes and won't be listed here.
    #if obs._remarks:
     # print("remarks:")
      #print("- "+obs.remarks("\n- "))

    print(estacion, hora, temp, fuerza_viento, dir_viento, lluvia)


LIPC Sun Dec  3 22:55:00 2017 17.0 7.4079936 260.0 0
LIPC Sun Dec  3 23:55:00 2017 17.0 7.4079936 270.0 0
LIPC Mon Dec  4 00:55:00 2017 17.0 9.259992 300.0 0
LIPC Mon Dec  4 01:55:00 2017 17.0 11.1119904 310.0 0
LIPC Mon Dec  4 02:55:00 2017 17.0 5.5559952 290.0 0
LIPC Mon Dec  4 03:55:00 2017 16.0 5.5559952 280.0 0
LIPC Mon Dec  4 04:55:00 2017 16.0 7.4079936 300.0 0
LIPC Mon Dec  4 05:55:00 2017 16.0 9.259992 270.0 0
LIPC Mon Dec  4 06:55:00 2017 16.0 9.259992 280.0 0
LIPC Mon Dec  4 07:55:00 2017 17.0 9.259992 300.0 0
LIPC Mon Dec  4 08:55:00 2017 17.0 7.4079936 300.0 0
LIPC Mon Dec  4 09:55:00 2017 19.0 11.1119904 310.0 0
LIPC Mon Dec  4 10:55:00 2017 20.0 3.7039968 20.0 0
LIPC Mon Dec  4 11:55:00 2017 20.0 9.259992 60.0 0
LIPC Mon Dec  4 12:55:00 2017 20.0 12.963988800000001 70.0 0
LIPC Mon Dec  4 13:55:00 2017 20.0 12.963988800000001 80.0 0
LIPC Mon Dec  4 14:55:00 2017 20.0 11.1119904 110.0 0
LIPC Mon Dec  4 15:55:00 2017 19.0 12.963988800000001 130.0 0
LIPC Mon Dec  4 16:55:00 

In [173]:
solo_metars

['METAR LIPC 062255Z 06013KT 9999 BKN035 20/16 Q1019 RMK BKN VIS MIN 9999 WIND THR11 06010KT WIND THR29 06010KT BLU',
 'METAR LIPC 062355Z 05008G18KT 9999 BKN035 20/16 Q1019 RMK BKN VIS MIN 9999 WIND THR11 05009KT WIND THR29 05012KT BLU',
 'METAR LIPC 070055Z 06014KT 9999 BKN040 21/16 Q1018 RMK BKN VIS MIN 9999 WIND THR11 05013KT WIND THR29 05014KT BLU',
 'METAR LIPC 070155Z 06012KT 9999 BKN040 21/16 Q1018 RMK BKN VIS MIN 9999 WIND THR11 06011KT WIND THR29 05012KT BLU',
 'METAR LIPC 070255Z 05012KT 9999 SCT040 21/16 Q1018 RMK BKN SCT080 VIS MIN 9999 WIND THR11 05011KT WIND THR29 04012KT BLU',
 'METAR LIPC 070355Z 05012KT 9999 SCT040 21/16 Q1018 RMK BKN SCT080 VIS MIN 9999 WIND THR11 05012KT WIND THR29 04011KT BLU',
 'METAR LIPC 070455Z 06015KT 9999 OVC040 21/17 Q1017 RMK OVC VIS MIN 9999 WIND THR11 06011KT WIND THR29 06013KT BLU',
 'METAR LIPC 070555Z 08012KT 9999 SCT045 21/17 Q1017 RMK OVC SCT070 VIS MIN 9999 WIND THR11 07010KT WIND THR29 06012KT BLU',
 'METAR LIPC 070655Z 03009KT 999

In [202]:
for element in solo_metars:
    
    obs = Metar.Metar(element)

    print(obs.time.date())
print(dia, obs.time.hour,obs.time.minute)

2017-12-06
2017-12-06
2017-12-07
2017-12-07
2017-12-07
2017-12-07
2017-12-07
2017-12-07
2017-12-07
2017-12-07
2017-12-07
2017-12-07
2017-12-07
2017-12-07
2017-12-07
2017-12-07
2017-12-07
2017-12-07
2017-12-07
2017-12-07
2017-12-07
2017-12-07
2017-12-07
2017-12-07
2016/09/07 21 55


In [55]:
for element in solo_metars:
    obs = Metar.Metar(element)
    test = obs.wind_dir.value()
    print(test)

220.0
220.0
0.0
300.0
320.0
0.0
0.0
250.0
240.0
260.0
320.0
340.0
310.0
330.0
330.0
320.0
100.0
110.0
140.0
160.0
170.0
230.0
230.0
230.0


In [62]:
def leer_meteo(url):
    headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_5)",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "accept-charset": "ISO-8859-1,utf-8;q=0.7,*;q=0.3",
    "accept-encoding": "gzip,deflate,sdch",
    "accept-language": "en-US,en;q=0.8",
    }

    #Metemos las cookies para que el html que devuelva sea el que necesitamos
    cook = dict(Prefs='FAVS:1|WXSN:1|PWSOBS:1|WPHO:1|PHOT:1|RADC:0|RADALL:0|HIST0:NULL|GIFT:1|SHOWMETAR:1|')
    
    r = requests.get(url, headers=headers, cookies=cook)
    
    if r.status_code != 200:
        print("request denied")
        print(r.status_code)
        sigue = 0

    soup = BeautifulSoup(r.text, 'html.parser')
    lista_info = []
    table =  soup.select("#obsTable")[0]

    for tr in table.find_all('tr'):
        for tds in tr.find_all('td', attrs={"colspan":"12"}):
            lista_info.append(tds.text)
            #print(tds.text)
    return lista_info

def filtrar_metars (lista):
    result = []
    lista.sort()
    
    for linea in lista:
        if linea[:5] == 'METAR':
            result.append(linea)
    return result

#No necesitamos esta funcion
def split_linea(lista):
    result = []
    for linea in lista:
        sline = linea.split()
        result.append(sline)
    return result

# No necesitamos esta funcion
def extraer_info(lista_de_listas, fecha):
    result = []
    for lista in lista_de_listas:
        estacion = lista[1]
        hora= lista[2]
        str_time_stamp = fecha+" "+str((int(hora[2:4])+2)%24)+":"+hora[4:6]+":00"
        viento = lista[3]
        temp = lista[5]
        if 'VRB' not in viento:
        #if len(viento)==7:
            result.append([estacion, str_time_stamp, int(viento[:3]),int(viento[3:5])*1.8519999983999, temp])
        else:
            print("------> Se deshecha: %s con datos: %s" %(hora,viento))
    return result


def lista_fecha(anio, mes, dia, dias):
    datelist = pd.date_range(datetime.datetime(anio, mes, dia), periods=dias).tolist()
    result = map(lambda x: x.to_pydatetime().date().strftime('%Y/%m/%d'), datelist)
    return list(result)


In [10]:
lista_df =[]
for dia in lista_fecha(anio_ini,mes_ini,dia_ini,num_dias):
    print(dia)
    url = 'https://www.wunderground.com/history/airport/'+aeropuerto+'/'+dia+'/DailyHistory.html'
    meteo_list = leer_meteo(url)
    solo_metars = filtrar_metars(meteo_list)
    split = split_linea(solo_metars)
    info = extraer_info(split, dia)

    dfinfo = pd.DataFrame(info)
    dfinfo.columns = ['Estacion','Fecha_Hora','Dir_Viento','Fuerza_Viento','Temp']
    dfinfo['Fecha_Hora'] = pd.to_datetime(dfinfo['Fecha_Hora'])
    lista_df.append(dfinfo)
dfresultado_final = pd.concat(lista_df)

dfresultado_final.to_excel("out.xls")


2017/09/05
2017/09/06
2017/09/07
2017/09/08
2017/09/09
2017/09/10
2017/09/11
2017/09/12
2017/09/13
2017/09/14
2017/09/15
2017/09/16
2017/09/17
2017/09/18
2017/09/19
2017/09/20
2017/09/21
2017/09/22
2017/09/23
2017/09/24
2017/09/25
2017/09/26
2017/09/27
2017/09/28
2017/09/29
2017/09/30
2017/10/01
2017/10/02
2017/10/03
2017/10/04


In [175]:

from __future__ import print_function

from metar import Metar

# A sample METAR report
#code = "METAR KEWR 111851Z VRB03G19KT 2SM R04R/3000VP6000FT TSRA BR FEW015 BKN040CB BKN065 OVC200 22/22 A2987 RMK AO2 PK WND 29028/1817 WSHFT 1812 TSB05RAB22 SLP114 FRQ LTGICCCCG TS OHD AND NW-N-E MOV NE P0013 T02270215"
code = 'METAR LIPC 042255Z 25002KT CAVOK 18/17 Q1018 RMK BKN BKN130 VIS MIN 9999 WIND THR11 25003KT WIND THR29 25004KT BLU'
print("-----------------------------------------------------------------------")
print("METAR: ",code)
print("-----------------------------------------------------------------------")

# Initialize a Metar object with the coded report
try:
    obs = Metar.Metar(code)
except:
    pass
# Print the individual data

# The 'station_id' attribute is a string.
print("station: %s" % obs.station_id)

if obs.type:
  print("type: %s" % obs.report_type())

# The 'time' attribute is a datetime object
if obs.time:
  print("time: %s" % obs.time.ctime())

# The 'temp' and 'dewpt' attributes are temperature objects
if obs.temp:
  print("temperature: %s" % obs.temp.string("C"))

if obs.dewpt:
  print("dew point: %s" % obs.dewpt.string("C"))

# The wind() method returns a string describing wind observations
# which may include speed, direction, variability and gusts.
if obs.wind_speed:
  print("wind: %s" % obs.wind())

# The peak_wind() method returns a string describing the peak wind 
# speed and direction.
if obs.wind_speed_peak:
  print("wind: %s" % obs.peak_wind())

# The visibility() method summarizes the visibility observation.
if obs.vis:
  print("visibility: %s" % obs.visibility())

# The runway_visual_range() method summarizes the runway visibility
# observations.
if obs.runway:
  print("visual range: %s" % obs.runway_visual_range())

# The 'press' attribute is a pressure object.
if obs.press:
  print("pressure: %s" % obs.press.string("mb"))

# The 'precip_1hr' attribute is a precipitation object.
if obs.precip_1hr:
  print("precipitation: %s" % obs.precip_1hr.string("in"))

# The present_weather() method summarizes the weather description (rain, etc.)
print("weather: %s" % obs.present_weather())

# The sky_conditions() method summarizes the cloud-cover observations.
print("sky: %s" % obs.sky_conditions("\n     "))

# The remarks() method describes the remark groups that were parsed, but 
# are not available directly as Metar attributes.  The precipitation, 
# min/max temperature and peak wind remarks, for instance, are stored as
# attributes and won't be listed here.
if obs._remarks:
  print("remarks:")
  print("- "+obs.remarks("\n- "))

print("-----------------------------------------------------------------------\n")


-----------------------------------------------------------------------
METAR:  METAR LIPC 042255Z 25002KT CAVOK 18/17 Q1018 RMK BKN BKN130 VIS MIN 9999 WIND THR11 25003KT WIND THR29 25004KT BLU
-----------------------------------------------------------------------
station: LIPC
type: routine report, cycle 23 (automatic report)
time: Mon Dec  4 22:55:00 2017
temperature: 18.0 C
dew point: 17.0 C
wind: WSW at 2 knots
visibility: 10000 meters
pressure: 1018.0 mb
weather: 
sky: 
-----------------------------------------------------------------------

